# Artificial Intel & Heurist Prg - Project 3

In [7]:
class HexapawnAI:
    def __init__(self):
        self.AI_PLAYER = "b"
        self.HUMAN_PLAYER = "w"

    # Terminal state evaluation
    def check_winner(self, board: "HexapawnBoard", player_to_move: str):
        b = board.board
        size = board.size

        # Promotion wins
        if "w" in b[0]:
            return "w"
        if "b" in b[size - 1]:
            return "b"

        # Legal moves for both sides
        white_moves = board.get_legal_moves("w")
        black_moves = board.get_legal_moves("b")

        # If it's white's turn and white has no moves -> black (other side) wins
        if player_to_move == self.HUMAN_PLAYER and not white_moves:
            return self.AI_PLAYER

        # If it's black's turn and black has no moves -> white wins
        if player_to_move == self.AI_PLAYER and not black_moves:
            return self.HUMAN_PLAYER

        return None  # no winner yet

    # Minimax search
    def minimax(self, board, maximizing):
        # Determine who is to move at this node
        player_to_move = self.AI_PLAYER if maximizing else self.HUMAN_PLAYER
        winner = self.check_winner(board, player_to_move)

        # Scoring
        if winner == self.AI_PLAYER:
            return +1, None
        if winner == self.HUMAN_PLAYER:
            return -1, None
    
        moves = board.get_legal_moves(player_to_move)
        if not moves:
            return (-1, None) if maximizing else (+1, None)

        if maximizing:
            best_score = -999
            best_move = None
            for move in moves:
                new_board = board.clone()
                new_board.make_move(move)
                score, _ = self.minimax(new_board, False)  # next node minimizing
                if score > best_score:
                    best_score = score
                    best_move = move
            return best_score, best_move

        else:  # minimizing
            best_score = 999
            best_move = None
            for move in moves:
                new_board = board.clone()
                new_board.make_move(move)
                score, _ = self.minimax(new_board, True)  # next node maximizing
                if score < best_score:
                    best_score = score
                    best_move = move
            return best_score, best_move

    # choose best move
    def get_best_move(self, board: "HexapawnBoard"):
        _, move = self.minimax(board, maximizing=True)
        return move


In [8]:
# Hexapawn Board Template
# Simple and smart starting point for a 3x3 hexapawn game board.

class HexapawnBoard:
    def __init__(self):
        # "w" = white pawn, "b" = black pawn, "." = empty
        self.size = 4
        self.board = [
            ["b", "b", "b", "b"],
            [".", ".", ".", "."],
            [".", ".", ".", "."],
            ["w", "w", "w", "w"],
        ]

    def display(self):
        for row in self.board:
            print(" ".join(row))
        print()

    def clone(self):
        # Helpful for AI search trees
        new_board = HexapawnBoard()
        new_board.board = [row[:] for row in self.board]
        return new_board

    def get_legal_moves(self, player):
        """
        Returns a list of moves for the given player.
        A move is represented as: (from_row, from_col, to_row, to_col)
        """
        moves = []
        direction = -1 if player == "w" else 1  # white moves up, black moves down

        for r in range(self.size):
            for c in range(self.size):
                if self.board[r][c] != player:
                    continue

                nr = r + direction  # next row in forward direction

                if 0 <= nr < self.size:
                    # Forward
                    if self.board[nr][c] == ".":
                        moves.append((r, c, nr, c))

                    # Capture left
                    if c - 1 >= 0:
                        if self.board[nr][c - 1] != "." and self.board[nr][c - 1] != player:
                            moves.append((r, c, nr, c - 1))

                    # Capture right
                    if c + 1 < self.size:
                        if self.board[nr][c + 1] != "." and self.board[nr][c + 1] != player:
                            moves.append((r, c, nr, c + 1))

        return moves

    def make_move(self, move):
        # Placeholder: apply (from_row, from_col, to_row, to_col)
        (fr, fc, tr, tc) = move
        self.board[tr][tc] = self.board[fr][fc]
        self.board[fr][fc] = "."

if __name__ == "__main__":
    board = HexapawnBoard()
    ai = HexapawnAI()

    while True:
        board.display()

        # Human move
        human_moves = board.get_legal_moves("w")
        if not human_moves:
            print("Black wins! (white has no moves)")
            break

        print("Your legal moves:", human_moves)
        move = tuple(map(int, input("Enter move: fr fc tr tc: ").split()))
        board.make_move(move)

        # Check winner
        winner = ai.check_winner(board, player_to_move="b")
        if winner:
            board.display()
            print("Winner:", winner)
            break

        # AI move
        ai_move = ai.get_best_move(board)
        print("AI moves:", ai_move)
        board.make_move(ai_move)

        # Check winner
        winner = ai.check_winner(board, player_to_move="w")
        if winner:
            board.display()
            print("Winner:", winner)
            break


b b b b
. . . .
. . . .
w w w w

Your legal moves: [(3, 0, 2, 0), (3, 1, 2, 1), (3, 2, 2, 2), (3, 3, 2, 3)]


Enter move: fr fc tr tc:  3 0 2 0


AI moves: (0, 0, 1, 0)
. b b b
b . . .
w . . .
. w w w

Your legal moves: [(3, 1, 2, 1), (3, 2, 2, 2), (3, 3, 2, 3)]


Enter move: fr fc tr tc:  3 1 2 1


AI moves: (0, 1, 1, 1)
. . b b
b b . .
w w . .
. . w w

Your legal moves: [(2, 0, 1, 1), (2, 1, 1, 0), (3, 2, 2, 2), (3, 3, 2, 3)]


Enter move: fr fc tr tc:  2 0 1 1


AI moves: (0, 2, 1, 2)
. . . b
b w b .
. w . .
. . w w

Your legal moves: [(1, 1, 0, 1), (2, 1, 1, 0), (2, 1, 1, 2), (3, 2, 2, 2), (3, 3, 2, 3)]


Enter move: fr fc tr tc:  1 1 0 1


. w . b
b . b .
. w . .
. . w w

Winner: w
